In [4]:
%reset -f
import numpy as np
import sys
from datetime import datetime
import os

In [5]:
output_log_file = "experiment_3_H.log"
log_file = open(output_log_file, 'a')
sys.stdout = log_file
print(output_log_file)
print(datetime.now().strftime("%a %b %d %H:%M:%S %Y"))

In [6]:
DEFAULT_WEIGHT_VALUE = 0  
image_mode = False
debug_mode = False

%run attack_graph_MIR100.ipynb

In [7]:
attack_rate_list = [0]

In [8]:
def random_steps(route, attack_rate=None, defense_rate=None, graph=None):
    
    # Part01: extract hardness values from all edges & append them
    hardness = []
    for i in range(len(route) - 1):
        start_node = route[i]
        end_node = route[i + 1]
        
        # Initialize vars for max weight loop
        weights = []
        # Collect weights for max
        for edge in graph[start_node][end_node].values():
            weights.append(edge.get('weight', DEFAULT_WEIGHT_VALUE))
        # Get maximum weight
        max_weight = max(weights) if weights else DEFAULT_WEIGHT_VALUE
        
        min_weights = []
        # Collect weights for min
        for edge in graph[start_node][end_node].values():
            min_weights.append(edge.get('weight', DEFAULT_WEIGHT_VALUE))
        # Get minimum weight
        min_weight = min(min_weights) if min_weights else DEFAULT_WEIGHT_VALUE
        
        hardness.append(np.exp(-min_weight))
       
    hardness = np.array(hardness)

    # print(f'Hardness: {hardness}')
    
    # Part02: Based on extracted hardness values

    cp = np.concatenate(([1.0], np.cumprod(hardness))) #cumulative probs

    stop_probs = np.concatenate((1 - hardness, [1.0]))

    # part03: generate final distribution
    
    pdf = cp * stop_probs

    sp = pdf.sum()
    # case where prob are essentially zero
    if sp < 1e-15:
        pdf = np.full_like(pdf, 1e-7)

    # Normalize 
    ret = pdf/ sp
    return ret

In [9]:
%run heuristic_defense.ipynb

In [10]:
sys.stdout = sys.__stdout__
log_file.close()
with open(output_log_file, 'r') as f:
    print(f.read())